In [1]:
import dicom2nifti
from rt_utils import RTStructBuilder

In [2]:
import nibabel as nib
import glob
import numpy as np
import pandas as pd
import os
import pydicom as pdcm

In [3]:
window_type = 'lung'
root_p = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/"
source_csv=root_p+"Users/Luis/CSVs/RegisteredACCTS.csv"
save_path = root_p+"Users/Luis/RRegistered_ACCTs_Nii/"

tot_labels = ['ITV',"IGTV","IgTV"]
tumor_labels = ['ITVtumor']
ln_labels = ['ITVklieren']

structure_for_mask = None
image_generation_flag=True
mask_generation_flag=True

In [6]:
df = pd.read_csv(source_csv)
#df.dropna(subset=['Px'], inplace=True)
df['Px'] = df['Px'].astype(int)
id_column = df.iloc[:, 0]
df

,Px,Path1,Path2,Path3,Path4
0,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,NaN
1,23840,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,NaN
2,110541,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN
3,248219,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,NaN
4,282824,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,NaN
5,624804,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,NaN
6,9385224,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,NaN
7,9521641,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,NaN,NaN
8,9593098,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,NaN
9,9829221,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,NaN


In [14]:
patientID = 10539
avgCT_dfPath = df[id_column==patientID]['CT'].iloc[0]
RTSTruct_dfPath = df[id_column==patientID]['Tot'].iloc[0]
rtdicom = pdcm.read_file(RTSTruct_dfPath)
print(rtdicom)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 198
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: RT Structure Set Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 2.16.840.1.113669.2.931128.289629209.20150513125112.505113
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.826.0.1.3680043.8.691.0.50
(0002, 0013) Implementation Version Name         SH: '1.2.0'
-------------------------------------------------
(0008, 0012) Instance Creation Date              DA: '20150513'
(0008, 0013) Instance Creation Time              TM: '125126'
(0008, 0016) SOP Class UID                       UI: RT Structure Set Storage
(0008, 0018) SOP Instance UID                    UI: 2.16.840.1.113669.2.931128.289629209.20150513125112.505113
(0008, 0020) Study Date                         

In [8]:
count=0
for patientID in id_column:
    #avgCT_dfPath = df[id_column==patientID]['CT'].iloc[0]
    #RTSTruct_dfPath = df[id_column==patientID]['Tot'].iloc[0]
    ACCTfast_dfPath1 = df[id_column==patientID]['Path1'].iloc[0]
    ACCTfast_dfPath2 = df[id_column==patientID]['Path2'].iloc[0]
    ACCTfast_dfPath3 = df[id_column==patientID]['Path3'].iloc[0]
    ACCTfast_dfPath4 = df[id_column==patientID]['Path4'].iloc[0]
    
    #PET_dfPath = df[id_column==patientID]['PET'].iloc[0]
    #ACCTBody_dfPath = df[id_column==patientID]['ACCTBody'].iloc[0]

    if not os.path.exists(save_path+str(patientID)):
        os.makedirs(save_path+str(patientID)+'/')
        print("NEW Px:",patientID,"Count",count)
    if len(os.listdir(save_path+str(patientID)))>=2:
        print("Already Complete Px:",patientID,"Count",count)
    else:
        print("Empty Folder Px:",patientID,"Count",count)
        save_px_path = os.path.join(save_path,str(patientID))
        #fpaths=[]
        #fpaths.append(ACCTfast_dfPath)
        #fpaths.append(avgCT_dfPath)
        #fpaths.append(RTSTruct_dfPath)
        #fpaths.append(PET_dfPath)
        #fpaths.append(ACCTBody_dfPath)    
        dicom2nifti.convert_directory(ACCTfast_dfPath1, save_px_path, compression=True, reorient=True)
        if not pd.isna(ACCTfast_dfPath2):
            dicom2nifti.convert_directory(ACCTfast_dfPath2, save_px_path, compression=True, reorient=True)
        if not pd.isna(ACCTfast_dfPath3):
            dicom2nifti.convert_directory(ACCTfast_dfPath3, save_px_path, compression=True, reorient=True)
        if not pd.isna(ACCTfast_dfPath4):
            dicom2nifti.convert_directory(ACCTfast_dfPath4, save_px_path, compression=True, reorient=True)
        
        files = os.listdir(save_px_path)

        if not len(files)>0:
            print("Not CT built")
        else:
            print("Built",files)  
            if True: #not len(RTSTruct_dfPath)>0:
                print("NO RT structs")
            else:
                ct_nifti = nib.load(os.path.join(save_px_path,files[0]))
                rtstructUTILS = RTStructBuilder.create_from(
                    dicom_series_path=avgCT_dfPath,
                    rt_struct_path=RTSTruct_dfPath.replace("\\\\", "/"))
                all_names = rtstructUTILS.get_roi_names()
                for curr_mask in tot_labels: 
                    for curr_name in all_names: 
                        if curr_mask == curr_name:
                            try:
                                mask_3d = rtstructUTILS.get_roi_mask_by_name(curr_mask)
                                saved_mask = curr_mask
                                print("Built RT for:",curr_mask)

                                mask_nifti = nib.Nifti1Image(mask_3d, affine=ct_nifti.affine, header=ct_nifti.header)
                                nib.save(mask_nifti, save_px_path+'/'+saved_mask+'.nii.gz')
                            except Exception as e:
                                print(e)
                                pass


                    
    count+=1 

Empty Folder Px: 10539 Count 0
Built ['1_acct_rregist001_fast.nii.gz', '1_pet_rregist001_fast.nii.gz']
NO RT structs
NEW Px: 23840 Count 1
Empty Folder Px: 23840 Count 1
Built ['1_acct_rregist001_fast.nii.gz', '1_acct_rregist001_fine.nii.gz']
NO RT structs
NEW Px: 110541 Count 2
Empty Folder Px: 110541 Count 2
Built ['1_acct_rregist001_fast.nii.gz', '1_acct_rregist001_fine.nii.gz', '6_pet_wb_earl.nii.gz']
NO RT structs
NEW Px: 248219 Count 3
Empty Folder Px: 248219 Count 3
Built ['1_acct_rregist001_fine.nii.gz', '1_pet_rrregist001_fine.nii.gz']
NO RT structs
NEW Px: 282824 Count 4
Empty Folder Px: 282824 Count 4
Built ['1_acct_rregist001_fine.nii.gz', '1_pet_rregist001_fine.nii.gz']
NO RT structs
NEW Px: 624804 Count 5
Empty Folder Px: 624804 Count 5
Built ['1_acct_rregist001_fast.nii.gz', '1_pet_rregist001_fine.nii.gz']
NO RT structs
NEW Px: 9385224 Count 6
Empty Folder Px: 9385224 Count 6


C:\Users\delaoArevaLR\AppData\Local\anaconda3\envs\Ch2_July23_2\lib\site-packages\pydicom\valuerep.py:443: UserWarning: Invalid value for VR UI: '1.3.12.2.1107.5.1.4.10001.'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)


Built ['1_acct_rregist001_fine.nii.gz', '1_per_rregist001_fine.nii.gz']
NO RT structs
NEW Px: 9521641 Count 7
Empty Folder Px: 9521641 Count 7
Built ['1_acct_rregist001_fine.nii.gz']
NO RT structs
NEW Px: 9593098 Count 8
Empty Folder Px: 9593098 Count 8
Built ['1_acct_rregist001_fine.nii.gz', '1_pet_rregist001_fine.nii.gz']
NO RT structs
NEW Px: 9829221 Count 9
Empty Folder Px: 9829221 Count 9
Built ['1_acct_rregist001_fast.nii.gz', '1_pet_rregist001_fine.nii.gz']
NO RT structs
NEW Px: 9854449 Count 10
Empty Folder Px: 9854449 Count 10
Built ['1_acct_rregist001_fine.nii.gz', '1_pet_rregist001_fast.nii.gz', '1_pet_rregist001_fine.nii.gz', '1_rregist001_fast.nii.gz']
NO RT structs
NEW Px: 9936840 Count 11
Empty Folder Px: 9936840 Count 11
Built ['1_pet_ar_9936840.nii.gz', '1_rregist001_fast.nii.gz', '1_rregist001_fine.nii.gz']
NO RT structs


In [ ]:
root_directory = save_path

# Iterate through all files in the root directory
for filename in os.listdir(root_directory):
    # Construct the full path to the file
    full_path = os.path.join(root_directory, filename)

    # Check if it's a file and not a directory
    if os.path.isfile(full_path):
        
        # Extract MatchID and PETID from the incorrect file name
        if filename[-10] == 'G':
            match_id, pet_id = filename[:-11],filename[-11:]
        else:
            match_id, pet_id = filename[:-10],filename[-10:]
        print(match_id, pet_id)
        
        # Create the correct directory path
        correct_directory = os.path.join(root_directory, match_id)

        # Create the correct file name
        correct_filename = pet_id

        # Create the full path for the correct location
        correct_full_path = os.path.join(correct_directory, correct_filename)
        
        #print("Path",full_path)
        #print("Correct_Path",correct_full_path)
        
        # Rename and move the file to the correct location
        os.rename(full_path, correct_full_path)
        

In [ ]:
os.listdir(root_directory):

In [ ]:
#Px: 1756631 Count 94: AttributeError: 'FileDataset' object has no attribute 'RadiopharmaceuticalInformationSequence'
avgCT_dfPath[0]

In [6]:
def convert_dicom_to_nifty(input_filepaths,rtstructUTILS,patientID,
                           output_folder,
                           labels_rtstruct,extension,filename,
                           pixel_spacing = None, image_position_patient=None,axial_positions=None,
                           np_image=None,dtype_image=np.float32,dtype_mask=np.uint8,):
    """Function to convert the dicom files contained in input_filepaths to one
       NIFTI image.

    Args:
        input_filepaths (list): list of the dicom paths
        output_folder (str): path to the output folder where to store the
                             NIFTI file.
        modality (str, optional): The modality of the DICOM, it is used to
                                  obtain the correct physical values
                                  (Hounsfield unit for the CT and SUV for the
                                  PT). Defaults to 'CT'.
        sitk_writer (sitk.WriteImage(), optional): The SimpleITK object used
                                                   to write an array to the
                                                   NIFTI format. Defaults to
                                                   None.
        rtstruct_file (list, optional): Path to the RTSTRUCT file associated
                                       with the current image. Defaults to
                                       None.
        labels_rtstruct (list, optional): List of label to extract from the
                                          RTSTRUCT. Defaults to ['GTVt'].
        patient_weight_from_ct (float, optional): If the patient's weight is
                                                  missing from the PT DICOM
                                                  it can be provided through
                                                  this argument. Defaults to
                                                  None.
        extension (str, optional): The extension in which to save the NIFTI.
                                   Defaults to '.nii'.
        dtype_image (numpy.dtype, optional): The dtype in which to save the
                                             image. Defaults to np.float32.
        dtype_mask (numpy.dtype, optional): The dtype in which to save the
                                            segmentation. Defaults to np.uint8.

    Raises:
        MissingWeightException: Error to alert when the weight is missing from
                                the PT, to compute the SUV.
        RuntimeError: Error to alert when one or more slices are missing
        ValueError: Raised when a modality or a unit (for the PT) is not
                    handled.

    Returns:
        numpy.array: The numpy image, used to compute the bounding boxes
    """

    slices = [pdcm.read_file(dcm) for dcm in input_filepaths]
    
    modality = slices[0].Modality
    print("working on: ",modality)
    
    if modality != 'RTSTRUCT':
        slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
        if modality == 'PT':
            if float(slices[0][Tag(0x00101030)].value) is None:
                if hasattr(slices[0], 'PatientWeight'):
                    patient_weight = float(slices[0].PatientsWeight)
                elif patient_weight_from_ct is not None:
                    patient_weight = patient_weight_from_ct
                else:
                    raise MissingWeightException(
                        'Cannot compute SUV the weight is missing')
            else:
                patient_weight = float(slices[0][Tag(0x00101030)].value)

        # Check if all the slices come from the same serie
        same_serie_uid = True
        serie_uid = slices[0].SeriesInstanceUID
        for s in slices:
            same_serie_uid *= serie_uid == s.SeriesInstanceUID

        if not same_serie_uid:
            print("RuntimeError('A slice comes from another serie")
            return 0, 0, None,None
            

        axial_positions = np.asarray([k.ImagePositionPatient[2] for k in slices])
        # Compute redundant slice positions
        ind2rm = [
            ind for ind in range(len(axial_positions))
            if axial_positions[ind] == axial_positions[ind - 1]
        ]
        # Check if there is redundancy in slice positions and remove them
        if len(ind2rm) > 0:
            slices = [k for i, k in enumerate(slices) if i not in ind2rm]
            axial_positions = np.asarray(
                [k.ImagePositionPatient[2] for k in slices])

        slice_spacing = (slices[1].ImagePositionPatient[2] -
                         slices[0].ImagePositionPatient[2])

        pixel_spacing = np.asarray([
            slices[0].PixelSpacing[0],
            slices[0].PixelSpacing[1],
            slice_spacing,
        ])

        if modality == 'CT':
            np_image = get_physical_values_ct(slices, dtype=dtype_image)
        elif modality == 'PT':
            np_image = get_physical_values_pt_new(slices,
                                              patient_weight,
                                              dtype=dtype_image)
        else:
            raise ValueError('The modality {} is not supported'.format(modality))

        position_final_slice = (
            len(slices) - 1) * slice_spacing + slices[0].ImagePositionPatient[2]
        # Test whether some slices are missing
        if not is_approx_equal(position_final_slice,
                               float(slices[-1].ImagePositionPatient[2])):
            if (position_final_slice - axial_positions[-1]) / slice_spacing < 1.5:
                # If only one slice is missing
                diff = np.asarray([
                    not is_approx_equal(
                        float(axial_positions[ind]) -
                        float(axial_positions[ind - 1]) - slice_spacing, 0)
                    for ind in range(1, len(axial_positions))
                ])
                ind2interp = int(np.where(diff)[0])
                new_slice = (np_image[:, :, ind2interp] +
                             np_image[:, :, ind2interp + 1]) * 0.5
                new_slice = new_slice[..., np.newaxis]
                np_image = np.concatenate(
                    (np_image[..., :ind2interp], new_slice, np_image[...,
                                                                     ind2interp:]),
                    axis=2)
                warnings.warn(
                    "One slice is missing, we replaced it by linear interpolation")
            else:
                # if more than one slice are missing
                raise RuntimeError('Multiple slices are missing')

        image_position_patient = [float(k) for k in slices[0].ImagePositionPatient]

        try:
            sitk_image = get_sitk_volume_from_np(np_image, pixel_spacing,
                                             image_position_patient)
        except:
            print("Axes don't match PET?")
            return 0,0,None,None

        #correct_patient_name(str(slices[0].PatientName))
        output_filepath = join(
                output_folder,
                patientID + '_' + filename + '_' +
                modality.lower() + extension)
        sitk_writer = sitk.ImageFileWriter()
        sitk_writer.SetImageIO('NiftiImageIO')
        sitk_writer.SetFileName(output_filepath)
        sitk_writer.Execute(sitk_image)
    if modality == 'RTSTRUCT':
        rtstruct_file = input_filepaths
        for rt_file in rtstruct_file:
            rt_utils = rtstructUTILS
            print("Converting RTSTRUCT image",len(rtstruct_file))

            masks = get_masks(rt_file,
                            labels=labels_rtstruct,
                            image_position_patient=image_position_patient,
                            axial_positions=axial_positions,
                            pixel_spacing=pixel_spacing,
                            shape=np_image.shape,
                            dtype=dtype_image,
                            rt_utils=rt_utils)
            for label, np_mask in masks: #label.lower()
                #print("Labels: ",label)
                output_filepath = join(output_folder,patientID + '_' +modality.lower() + extension)
                #output_filepath = join(output_folder,filename+'_'+modality.lower() + extension)
                sitk_writer = sitk.ImageFileWriter()
                sitk_writer.SetImageIO('NiftiImageIO')                
                output_filepath_mask = output_filepath.split('.')[0] + '_' + label + extension
                #print(output_filepath_mask)
                
                sitk_mask = get_sitk_volume_from_np(np_mask, pixel_spacing,image_position_patient)
                
                sitk_writer.SetFileName(output_filepath_mask)
                sitk_writer.Execute(sitk_mask)

    #return np.transpose(np_image,(1, 0, 2)), pixel_spacing, image_position_patient
    return np.transpose(np_image,(1, 0, 2)), pixel_spacing, image_position_patient,axial_positions


def get_sitk_volume_from_np(np_image, pixel_spacing, image_position_patient):
    trans = (2, 0, 1)
    sitk_image = sitk.GetImageFromArray(np.transpose(np_image, trans))
    sitk_image.SetSpacing(pixel_spacing)
    sitk_image.SetOrigin(image_position_patient)
    return sitk_image


def is_approx_equal(x, y, tolerance=0.05):
    return abs(x - y) <= tolerance


class MissingWeightException(RuntimeError):
    pass


def correct_patient_name(patient_name):
    output = patient_name.replace("HN-CHUS-", "CHUS")
    output = output.replace("HN-CHUM-", "CHUM")
    output = output.replace("HN-HGJ-", "CHGJ")
    output = output.replace("HN-HMR-", "CHMR")
    output = output.replace("HN-CHUV-", "CHUV")
    return output


def get_masks(rtstruct_file,
              labels, #=['GTV tumor','GTV-PT', 'GTV  primaire tumor', 'gtv goed',  'GTV_CT+PET',  'GTVTumor',  'GTVtumor', 'GTV70',  'GTV_tumor','GTV PET+CT',  'gtv', 'gtv PT', 'GTV PT', 'gtv tumor', 'GTV',  'GTVpt'],
              image_position_patient=None,
              axial_positions=None,
              pixel_spacing=None,
              shape=None,
              dtype=np.int8,
              rt_utils=None):
    print('in get_mask')
    contours = read_structure(rtstruct_file, labels=labels)
    return get_mask_from_contour(contours,
                                 image_position_patient,
                                 axial_positions,
                                 pixel_spacing,
                                 shape,
                                 dtype=dtype,
                                 rt_utils=rt_utils)

#labels=['GTV tumor','GTV-PT', 'GTV  primaire tumor', 'gtv goed',  'GTV_CT+PET',  'GTVTumor',  'GTVtumor', 'GTV70',  'GTV_tumor','GTV PET+CT',  'gtv', 'gtv PT', 'GTV PT', 'gtv tumor', 'GTV',  'GTVpt']

def read_structure(rtstruct_file, labels):#=['gtvp', 'gtv tumor', 'gtv_pt','gtv-tumor','gtv-pt','gtvprim_tumor','gtv','gtvpt','gtv pt','gtv_tumor','gtvprimairetumor','gtv primaire tumor','gtv  primaire tumor','gtvtumor','gtv_p'
    structure = pdcm.read_file(rtstruct_file)
    contours = []

    try:
        for i, roi_seq in enumerate(structure.StructureSetROISequence):
            contour = {}
            #print("Names: ",roi_seq.ROIName)
            for label in labels:            
                if roi_seq.ROIName.lower() == label.lower():
                    contour['color'] = structure.ROIContourSequence[i].ROIDisplayColor
                    contour['number'] = structure.ROIContourSequence[i].ReferencedROINumber
                    contour['name'] = roi_seq.ROIName
                    assert contour['number'] == roi_seq.ROINumber
                    try:
                        contour['contours'] = [s.ContourData for s in structure.ROIContourSequence[i].ContourSequence]
                        contours.append(contour)
                    except AttributeError:
                        print("AttributeError in the Contour, not in the RT file")
                        continue
    except AttributeError:
        print("AttributeError, no StructureROISequence in RT")
        return contours
    return contours

def read_headers(rtstruct_file, labels=['GTV tumor','GTV-PT', 'GTV  primaire tumor', 'gtv goed',  'GTV_CT+PET',  'GTVTumor',  'GTVtumor', 'GTV70',  'GTV_tumor','GTV PET+CT',  'gtv', 'gtv PT', 'GTV PT', 'gtv tumor', 'GTV',  'GTVpt',  'GTVp']):
    structure = pdcm.read_file(rtstruct_file)
    gtv_names = []
    try:
        for i, roi_seq in enumerate(structure.StructureSetROISequence):
            
            if  'gtv' in roi_seq.ROIName.lower() or 'primaire' in roi_seq.ROIName.lower() or 'lk' in roi_seq.ROIName.lower() or 'klie' in roi_seq.ROIName.lower() or 'ln' in roi_seq.ROIName.lower() or 'ctv' in roi_seq.ROIName.lower():

                gtv_names.append(roi_seq.ROIName)
            #gtv_names = gtv_names + [j for j in list(roi_seq.ROIName) if 'GTV' in j or 'gtv' in j]

        return list(set(gtv_names))
    except:
        return list()

def choose_rtstruct(rtstruct_file):

    for name in rtstruct_file:

        #print(name)

        structure = pdcm.read_file(name)

        try:
            for i, roi_seq in enumerate(structure.StructureSetROISequence):
                print(roi_seq.ROIName)
                if  'gtv' in roi_seq.ROIName.lower():
                #if  ('gtvpt' in roi_seq.ROIName.lower()) or ('gtv pt' in roi_seq.ROIName.lower()) or ('gtv-pt' in roi_seq.ROIName.lower()) or ('gtv  primaire tumor' in roi_seq.ROIName.lower()):
                    print('Found name ' + roi_seq.ROIName)
                    return [name]

            for i, roi_seq in enumerate(structure.StructureSetROISequence):
                print(roi_seq.ROIName)
                if  'primaire' in roi_seq.ROIName.lower():
                #if  ('gtvpt' in roi_seq.ROIName.lower()) or ('gtv pt' in roi_seq.ROIName.lower()) or ('gtv-pt' in roi_seq.ROIName.lower()) or ('gtv  primaire tumor' in roi_seq.ROIName.lower()):
                    print('Found name ' + roi_seq.ROIName)
                    return [name]
        except:
            print('Problem with RTSTRUCT')
            continue
            
    return list()

        #print(roi_seq.ROIName)
        #for label in labels:
            
            #print(roi_seq.ROIName)


def get_mask_from_contour(contours,
                          image_position_patient,
                          axial_positions,
                          pixel_spacing,
                          shape,
                          dtype=np.uint8,
                          rt_utils=None):

    print('get_mask_from_c')
    z = np.asarray(axial_positions)
    pos_r = image_position_patient[1]
    spacing_r = pixel_spacing[1]
    pos_c = image_position_patient[0]
    spacing_c = pixel_spacing[0]

    output = []
    for con in contours:
        try:
            mask_3d = rt_utils.get_roi_mask_by_name(con['name'])  
            output.append((con['name'], mask_3d.astype(int)))
            print("converted: ",con['name'])
        except Exception as e:
            print(e)
            continue
    return output


def get_physical_values_ct(slices, dtype=np.float32):
    image = list()
    for s in slices:
        image.append(
            float(s.RescaleSlope) * s.pixel_array + float(s.RescaleIntercept))
    return np.stack(image, axis=-1).astype(dtype)


def get_physical_values_pt_old(slices):
    units = slices[0].Units
    s = slices[0]
    if units == 'BQML':
        datetime_acquisition = datetime.strptime(
            s[Tag(0x00080022)].value + s[Tag(0x00080032)].value.split('.')[0],
            "%Y%m%d%H%M%S")
        datetime_serie = datetime.strptime(
            s[Tag(0x00080021)].value + s[Tag(0x00080031)].value.split('.')[0],
            "%Y%m%d%H%M%S")
        if datetime_serie < datetime_acquisition and datetime_serie > datetime(
                1950, 1, 1):
            pass
        else:
            pass
        return get_suv_from_bqml(slices)
    elif units == 'CNTS':
        return get_suv_philips(slices)
    else:
        raise ValueError('The {} units is not handled'.format(units))

def get_physical_values_pt_new(slices, patient_weight, dtype=np.float32):
    s = slices[0]
    units = s.Units

    if units == 'BQML':

        start_time_str = s.RadiopharmaceuticalInformationSequence[0].RadiopharmaceuticalStartTime
        start_time = time(int(start_time_str[0:2]),
                            int(start_time_str[2:4]),
                            int(start_time_str[4:6]))
        
        scan_time = datetime.strptime(
                    s[Tag(0x00080021)].value + s[Tag(0x00080031)].value.split('.')[0],
                    "%Y%m%d%H%M%S")
        start_datetime = datetime.combine(scan_time.date(), start_time)
        
        decay_time = (scan_time-start_datetime).total_seconds()

        return get_suv_from_bqml_new(slices,
                                 decay_time,
                                 patient_weight,
                                 dtype=dtype)

def get_suv_from_bqml_new(slices, decay_time, patient_weight, dtype=np.float32):
    # Get SUV from raw PET
    image = list()
    for s in slices:
        pet = float(s.RescaleSlope) * s.pixel_array + float(s.RescaleIntercept)
        half_life = float(
            s.RadiopharmaceuticalInformationSequence[0].RadionuclideHalfLife)
        total_dose = float(
            s.RadiopharmaceuticalInformationSequence[0].RadionuclideTotalDose)
        decay = 2**(-decay_time / half_life)
        actual_activity = total_dose * decay

        im = pet * patient_weight * 1000 / actual_activity
        image.append(im)
    return np.stack(image, axis=-1).astype(dtype)


def get_physical_values_pt(slices, patient_weight, dtype=np.float32):
    s = slices[0]
    units = s.Units
    if units == 'BQML':

        acquisition_datetime = datetime.strptime(
            s[Tag(0x00080022)].value + s[Tag(0x00080032)].value.split('.')[0],
            "%Y%m%d%H%M%S")
        serie_datetime = datetime.strptime(
            s[Tag(0x00080021)].value + s[Tag(0x00080031)].value.split('.')[0],
            "%Y%m%d%H%M%S")

        try:
            if (serie_datetime <= acquisition_datetime) and (
                    serie_datetime > datetime(1950, 1, 1)):
                scan_datetime = serie_datetime
            else:
                scan_datetime_value = s[Tag(0x0009100d)].value
                if isinstance(scan_datetime_value, bytes):
                    scan_datetime_str = scan_datetime_value.decode(
                        "utf-8").split('.')[0]
                elif isinstance(scan_datetime_value, str):
                    scan_datetime_str = scan_datetime_value.split('.')[0]
                else:
                    raise ValueError(
                        "The value of scandatetime is not handled")
                scan_datetime = datetime.strptime(scan_datetime_str,
                                                  "%Y%m%d%H%M%S")

            start_time_str = s.RadiopharmaceuticalInformationSequence[
                0].RadiopharmaceuticalStartTime
            start_time = time(int(start_time_str[0:2]),
                              int(start_time_str[2:4]),
                              int(start_time_str[4:6]))
            start_datetime = datetime.combine(scan_datetime.date(), start_time)
            decay_time = (scan_datetime - start_datetime).total_seconds()
        except KeyError:
            warnings.warn("Estimation of time decay for SUV"
                          " computation from average parameters")
            decay_time = 1.75 * 3600  # From Martin's code
        return get_suv_from_bqml(slices,
                                 decay_time,
                                 patient_weight,
                                 dtype=dtype)

    elif units == 'CNTS':
        return get_suv_philips(slices, dtype=dtype)
    else:
        raise ValueError('The {} units is not handled'.format(units))



def get_suv_philips(slices, dtype=np.float32):
    image = list()
    suv_scale_factor_tag = Tag(0x70531000)
    for s in slices:
        im = (float(s.RescaleSlope) * s.pixel_array +
              float(s.RescaleIntercept)) * float(s[suv_scale_factor_tag].value)
        image.append(im)
    return np.stack(image, axis=-1).astype(dtype)


def get_suv_from_bqml(slices, decay_time, patient_weight, dtype=np.float32):
    # Get SUV from raw PET
    image = list()
    for s in slices:
        pet = float(s.RescaleSlope) * s.pixel_array + float(s.RescaleIntercept)
        half_life = float(
            s.RadiopharmaceuticalInformationSequence[0].RadionuclideHalfLife)
        total_dose = float(
            s.RadiopharmaceuticalInformationSequence[0].RadionuclideTotalDose)
        decay = 2**(-decay_time / half_life)
        actual_activity = total_dose * decay

        im = pet * patient_weight * 1000 / actual_activity
        image.append(im)
    return np.stack(image, axis=-1).astype(dtype)


In [ ]:
save_px_path

In [ ]:
tempRTFilePath = os.path.join(RTSTruct_dfPath, os.listdir(RTSTruct_dfPath)[0])
tempRTFilePath